In [1]:
from tensorflow import keras #고수준 API
(train_input, train_target), (test_input, test_target) =\
  keras.datasets.fashion_mnist.load_data()

#입력층: 784개의 뉴런
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0 #픽셀값을 0~255 범위에서 0~1 사이로 변환
print(train_scaled.shape)
train_scaled = train_scaled.reshape(-1, 28*28) #28X28 크기의 2차원 배열을 784 크기의 1차원 배열로 펼친다
#x.reshape(4,5)= x를 (4,5) 2차원으로 바꾼 배열을 리턴한다. /(2,2,2) -> 3차원
#빠짐없이 배치될 수 있도록 '-1'이 들어가 있는 '행(row)' 의 개수가 가변적으로 정해짐
print(train_scaled.shape)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

4422102/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000, 784)


In [ ]:
from sklearn.preprocessing import StandardScaler
from tensorflow import keras #고수준 API
(train_input, train_target), (test_input, test_target) =\
  keras.datasets.fashion_mnist.load_data()

train_input = train_input.reshape(-1, 28*28)
sc = StandardScaler()
train_scaled = sc.fit_transform(train_input)

In [ ]:
#층 추가 -> 입력층과 출력층 사이에 층 = 은닉층 -> 활성화 함수 적용 = 비선형적으로 비틀어주기 위해 => 시그모이드 함수
#분류 문제는 클래스에 대한 확률 출력 = 활성화 함수 사용 = 시그모이드, 소프트맥스
#회귀의 출력은 임의의 수 = 활성화 함수 적용할 필요 없다.

#은닉층, 100개의 뉴런을 가진 밀집층 -> 기준이 없다, 경험에 의해, 출력층의 뉴런보다 많아야 한다
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden') #활성화 함수 = 시그모이드 / input_shape = 입력의 크기 = (784,)

#출력층, 10개 클레스 분류 -> 10개 뉴런
dense2 = keras.layers.Dense(10, activation='softmax', name='output')

#심층 신경망 만들기 -> 층을 추가해 연속적인 학습 -> 성능 좋아짐
model = keras.Sequential([dense1, dense2], name='패션 MNIST 모델') #은닉층, 출력층 순서
model.summary() #첫 번째 = 샘플(미니배치 경사 하강법), 두 번째 = 은닉층 뉴런 개수 / param = 픽셀 784개와 100개의 모든 조합에 대한 가중치 + 뉴런마다 1개 절편
#입력 픽셀 784개와 뉴런 100개 + 뉴런마다 1개 절편 = 78500개 모델 파라미터
#은닉층 뉴런 100개와 뉴런 10개 + 뉴런마다 1개 절편 = 1010개 모델 파라미터

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#층을 추가하는 다른 방법
model = keras.Sequential([keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
                          keras.layers.Dense(10, activation='softmax', name='output')], name='패션 MNIST 모델')
print(model.summary())
print()

model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))
print(model.summary())

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                        

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5645 - accuracy: 0.8064
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4111 - accuracy: 0.8509
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3760 - accuracy: 0.8628
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3532 - accuracy: 0.8716
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3352 - accuracy: 0.8791


In [6]:
#시그모이드 함수 = 그래프가 누워있어 올바른 출력을 만드는데 신속한 대응이 어렵다. -> 심층 신경망일수록 학습을 어렵게 만든다.
#렐루 이미지 처리에서 좋은 성능
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28))) #Flatten() = 입력 차원을 일렬로 펼치는 역할
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))
model.summary()


#입력층 784개의 뉴런
from sklearn.model_selection import train_test_split
from tensorflow import keras #고수준 API
(train_input, train_target), (test_input, test_target) =\
  keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0 #픽셀값을 0~255 범위에서 0~1 사이로 변환
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

model.evaluate(val_scaled, val_target)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 dense_8 (Dense)             (None, 100)               78500     
                                                                 
 dense_9 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5243 - accuracy: 0.8125
Epoch 2/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3900 - accuracy: 0.8589
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3

[0.3533652424812317, 0.878083348274231]

In [ ]:
#케라스는 다양한 종류의 경사 하강법을 제공한다 = 옵티마이저 / 기본 = RMSprop
#가장 기본적인 확률적 경사 하강법 = SGD -> 기본적으로 미니배치
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

#sgd = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True) 학습률 기본은 0.01 / 네스트로프 모맨텀은 모멘텀 최적화 2번 반복
#model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy') #'sgd'는 자동으로 SGD클래스 객체를 만들어 준다


#adagrad = keras.optimizers.Adagrad()
#model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

#rmsprop = keras.optimizers.RMSprop()
#model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

#adam은 모멘텀 최적화와 RMSprop의 장점을 접목 / RMSprop와 맨처음 시도해 볼 수 있는 알고리즘 /
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28))) #Flatten() = 입력 차원을 일렬로 펼치는 역할
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)
model.evaluate(val_scaled, val_target)

Epoch 1/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5311 - accuracy: 0.8150
Epoch 2/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3968 - accuracy: 0.8568
Epoch 3/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3522 - accuracy: 0.8725
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3231 - accuracy: 0.8817
Epoch 5/5
375/375 [==============================] - 1s 3ms/step - loss: 0.3360 - accuracy: 0.8787


[0.3360069692134857, 0.8786666393280029]